In [1]:
# Importando as Bibliotecas
import pandas as pd
import plotly.express as px

In [2]:
# Importando o DataFrame do site FiveThityEight
df = pd.read_csv("https://projects.fivethirtyeight.com/soccer-api/club/spi_matches.csv")
df = df[(df.league == "Brasileiro Série A") & (df.season == 2022)]
df = df[['date', 'team1', 'team2', 'proj_score1', 'proj_score2', 'xg1', 'xg2', 'score1', 'score2']]
df.reset_index(inplace=True, drop=True)
df.index = df.index.set_names(['Nº'])
df = df.rename(index=lambda x: x + 1)

# Criando as colunas de Pontos para determinar a Classificação
df.loc[(df['score1'] > df['score2']), 'Pontos_H'] = 3
df.loc[(df['score1'] == df['score2']), 'Pontos_H'] = 1
df.loc[(df['score1'] < df['score2']), 'Pontos_H'] = 0

df.loc[(df['score1'] < df['score2']), 'Pontos_A'] = 3
df.loc[(df['score1'] == df['score2']), 'Pontos_A'] = 1
df.loc[(df['score1'] > df['score2']), 'Pontos_A'] = 0

# Visualizando o DataFrame
df.head()

,date,team1,team2,proj_score1,proj_score2,xg1,xg2,score1,score2,Pontos_H,Pontos_A
Nº,,,,,,,,,,,
1,2022-04-09,Fluminense,Santos,1.40,0.93,1.75,0.10,0.0,0.0,1.0,1.0
2,2022-04-09,Atlético Goianiense,Flamengo,0.94,1.90,1.43,1.52,1.0,1.0,1.0,1.0
3,2022-04-09,Palmeiras,Ceará,2.04,0.85,2.72,1.69,2.0,3.0,0.0,3.0
4,2022-04-10,Coritiba,Goiás,1.63,1.24,1.64,0.76,3.0,0.0,3.0,0.0
5,2022-04-10,Atletico Mineiro,Internacional,1.94,0.97,2.35,0.44,2.0,0.0,3.0,0.0


In [3]:
# Classificação
Pontos_Home = df[['team1','Pontos_H']].groupby('team1').sum()
Pontos_Away = df[['team2','Pontos_A']].groupby('team2').sum()
Classificacao = pd.concat([Pontos_Home, Pontos_Away],axis=1)
Classificacao['Pontos'] = Classificacao.Pontos_H + Classificacao.Pontos_A
Classificacao = Classificacao[['Pontos']]
Classificacao.index = Classificacao.index.set_names(['Time'])
Classificacao = Classificacao.sort_values(['Pontos'], ascending=False)
Classificacao = Classificacao.rename_axis('index').reset_index()
Classificacao.reset_index(inplace=True, drop=True)
Classificacao.index = Classificacao.index.set_names(['Nº'])
Classificacao = Classificacao.rename(index=lambda x: x + 1)
Classificacao.columns = ['Time','Pontos']
Classificacao

,Time,Pontos
Nº,,
1,Palmeiras,54.0
2,Internacional,46.0
3,Flamengo,45.0
4,Fluminense,45.0
5,Corinthians,44.0
6,Atlético Paranaense,43.0
7,Atletico Mineiro,40.0
8,América Mineiro,36.0
9,Goiás,36.0


In [4]:
home_data = df[['team1', 'proj_score1', 'xg1', 'score1']]
away_data = df[['team2', 'proj_score2', 'xg2', 'score2']]

home_groupped = home_data.groupby(['team1']).mean().reset_index()
away_groupped = away_data.groupby(['team2']).mean().reset_index()

In [5]:
df0 = pd.merge(home_groupped, away_groupped, left_on=['team1'], right_on=['team2'])
df0.drop(['team2'], axis='columns', inplace=True)
df0.columns = ['Team','xG_pre_H','xG_H','Goals_H','xG_pre_A','xG_A','Goals_A']
df0.reset_index(inplace=True, drop=True)
df0.index = df0.index.set_names(['Nº'])
df0 = df0.rename(index=lambda x: x + 1)
df0

,Team,xG_pre_H,xG_H,Goals_H,xG_pre_A,xG_A,Goals_A
Nº,,,,,,,
1,América Mineiro,1.283684,1.230769,1.230769,0.971579,0.863846,0.461538
2,Atletico Mineiro,1.743684,1.543846,1.461538,1.400000,1.518462,1.153846
3,Atlético Goianiense,1.194737,1.397692,1.000000,0.890526,0.866923,0.769231
4,Atlético Paranaense,1.361579,1.477500,1.500000,1.031053,1.181429,0.928571
5,Avaí,1.201579,1.456154,1.384615,0.894737,1.019231,0.538462
6,Botafogo,1.238947,1.019231,0.769231,0.936842,1.128462,1.153846
7,Bragantino,1.511053,1.556923,1.615385,1.210526,1.188462,1.153846
8,Ceará,1.337368,0.987692,1.000000,0.994737,1.240000,1.000000
9,Corinthians,1.334211,1.168462,1.307692,1.063158,1.089231,1.000000


In [6]:
df = pd.merge(Classificacao, df0, left_on=['Time'], right_on=['Team'])
df = df[['Team','Pontos','xG_pre_H','xG_H','Goals_H','xG_pre_A','xG_A','Goals_A']]
df.columns = ['Team','Points','xG_pre_H','xG_H','Goals_H','xG_pre_A','xG_A','Goals_A']
df.reset_index(inplace=True, drop=True)
df.index = df.index.set_names(['Nº'])
df = df.rename(index=lambda x: x + 1)
df

,Team,Points,xG_pre_H,xG_H,Goals_H,xG_pre_A,xG_A,Goals_A
Nº,,,,,,,,
1,Palmeiras,54.0,1.857368,2.179231,1.923077,1.501053,1.300000,1.384615
2,Internacional,46.0,1.614737,2.164615,2.153846,1.293684,1.125385,1.000000
3,Flamengo,45.0,1.865789,2.210000,2.166667,1.572632,1.297143,1.071429
4,Fluminense,45.0,1.465263,1.517857,1.928571,1.166842,0.930833,1.083333
5,Corinthians,44.0,1.334211,1.168462,1.307692,1.063158,1.089231,1.000000
6,Atlético Paranaense,43.0,1.361579,1.477500,1.500000,1.031053,1.181429,0.928571
7,Atletico Mineiro,40.0,1.743684,1.543846,1.461538,1.400000,1.518462,1.153846
8,América Mineiro,36.0,1.283684,1.230769,1.230769,0.971579,0.863846,0.461538
9,Goiás,36.0,1.268421,1.268462,1.307692,0.932105,0.928462,0.923077


In [7]:
fig = px.bar(df, x='Team', y=['xG_pre_H','xG_H','Goals_H'], barmode="group")
fig.show()

In [8]:
fig = px.bar(df, x='Team', y=['xG_pre_A','xG_A','Goals_A'], barmode="group")
fig.show()